# In this file I want to test out the sparse matrix methods to figure out how I would implemement them and the speed boost they will give me

In [ ]:
import scipy.sparse as sparse
import scipy.sparse.linalg as spalg
import scipy.linalg as lalg
import numpy as np
import timeit
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
sigma_x = np.matrix('0. 1.; 1. 0.')
sigma_z = np.matrix('1. 0.; 0. -1.')
sparse_sigma_x = sparse.csr_matrix(sigma_x)
sparse_sigma_z = sparse.csr_matrix(sigma_z)

def sigma_zi(i,N):
    '''This returns the equivalent sigma_z for the ith qubit in the new Hilbert space given that there are N qubits'''
    return np.kron(np.eye(2**i), np.kron(sigma_z,np.eye(2**(N-(i+1)))))

def sparse_sigma_zi(i,N):
    return sparse.kron(sparse.identity(2**i, format = 'csr'), sparse.kron(sparse_sigma_z,sparse.identity(2**(N-(i+1)), format = 'csr')))

def sigma_xi(i,N):
    '''This returns the equivalent sigma_z for the ith qubit in the new Hilbert space given that there are N qubits'''
    return np.kron(np.eye(2**i), np.kron(sigma_x,np.eye(2**(N-(i+1)))))

def sparse_sigma_xi(i,N):
    return sparse.kron(sparse.identity(2**i, format = 'csr'), sparse.kron(sparse_sigma_x,sparse.identity(2**(N-(i+1)), format = 'csr')))


In [ ]:
sparse_sigma_zi(3,5)

In [ ]:
qubits = 9
h = np.random.random(qubits)
J = np.random.random((qubits,qubits))


def hamiltonian_parts():
        '''
        - Constructs the hamiltonian from the parameters input
        - If we are doing an anneal run where we have different annealing schedules for each qubits it will return a list 
        of hamiltonian parts for the base hamiltonian. Else it returns a single hamiltonian for this part
        '''
        #initiate the hamiltonians giving them the right dimensions
        Hb = []
        Hp = np.matrix(np.eye(2**qubits)*0.)
        # loop through adding pieces to construct the hamiltonians 
        for i in range(qubits):
            Hb.append(sigma_xi(i,qubits))
            Hp += h[i]*sigma_zi(i,qubits)
            # the following part adds the interaction between all qubits. There is 1 interaction coefficient for each pairing 
            for j in range(i+1,qubits): 
                Hp +=J[i][j]*sigma_zi(i,qubits)*sigma_zi(j,qubits)    
        Hb = sum(Hb)
        return Hb, Hp

def sparse_hamiltonian_parts():
        '''
        - Constructs the hamiltonian from the parameters input
        - If we are doing an anneal run where we have different annealing schedules for each qubits it will return a list 
        of hamiltonian parts for the base hamiltonian. Else it returns a single hamiltonian for this part
        '''
        #initiate the hamiltonians giving them the right dimensions
        Hb = []
        Hp = sparse.csr_matrix((2**qubits, 2**qubits), dtype=np.int8)
        # loop through adding pieces to construct the hamiltonians 
        for i in range(qubits):
            Hb.append(sparse_sigma_xi(i,qubits))
            Hp += h[i]*sparse_sigma_zi(i,qubits)
            # the following part adds the interaction between all qubits. There is 1 interaction coefficient for each pairing 
            for j in range(i+1,qubits): 
                Hp +=J[i][j]*sparse_sigma_zi(i,qubits)*sparse_sigma_zi(j,qubits)   
        Hb = sum(Hb)
        return Hb, Hp

In [ ]:
# lets see how the number of qubits affects the ratio in time taken
Qmax =12
Tn = np.zeros(Qmax)
Ts = np.zeros(Qmax)
for qubits in range(1,Qmax+1):
    h = np.random.random(qubits)
    J = np.random.random((qubits,qubits))
    Tn[qubits-1] = timeit.timeit(hamiltonian_parts,number = 3)
    Ts[qubits-1] = timeit.timeit(sparse_hamiltonian_parts,number = 3)
plt.figure()
plt.title('Time taken to construct the Hamiltonian')
plt.plot(range(1,Qmax+1), Tn, label = 'normal')
plt.plot(range(1,Qmax+1), Ts, label = 'sparse')
plt.xlabel('Qubits')
plt.ylabel('Time Taken')
plt.legend(framealpha= 0, loc = 'best')
plt.show()

We can see that there is a dramatic difference just in creating the Hamiltonian in the two methods. I should have found this earlier

In [ ]:
Qmax = 12
Hb0 = []
Hp0 = []
Hs0 = []
sparse_Hb0 = []
sparse_Hp0 = []
sparse_Hs0 = []
v0 = []

for i in range(1,Qmax+1):
    qubits = i
    h = np.random.random(qubits)
    J = np.random.random((qubits,qubits))

    hb,hp = hamiltonian_parts()
    shb,shp = sparse_hamiltonian_parts()
    hs = 0.3*hb+0.7*hp
    shs = 0.3*shb+0.7*shp
    
    Hb0.append(hb)
    Hp0.append(hp)
    Hs0.append(hs)
    sparse_Hb0.append(shb)
    sparse_Hp0.append(shp)
    sparse_Hs0.append(shs)
    
    v = np.matrix(np.random.random(2**qubits)).getH()
    v=v/(v.getH()*v).item(0)**0.5
    v0.append(v)

def diagal():
    d,P = lalg.eigh(Hs)
    return d,P
    
def sparse_diagal():
    d,P = spalg.eigsh(sparse_Hs,k=2**qubits-1)
    return d,P


In [ ]:
# lets see how the number of qubits affects the ratio in time taken
Qmax = 12
Tn = np.zeros(Qmax)
Ts = np.zeros(Qmax)
for qubits in range(3,Qmax+1):
    Hs = Hs0[qubits-1]
    sparse_Hs = sparse_Hs0[qubits-1]
    Tn[qubits-1] = timeit.timeit(diagal,number = 10)
    Ts[qubits-1] = timeit.timeit(sparse_diagal,number = 10)
plt.figure()
plt.title('Time taken for Diagonalisation')
plt.plot(range(1,Qmax+1), Tn, label = 'normal')
plt.plot(range(1,Qmax+1), Ts, label = 'sparse')
plt.xlabel('Qubits')
plt.ylabel('Time Taken')
plt.legend(framealpha= 0, loc = 'best')
plt.show()

In [ ]:
#print spalg.eigsh(sparse_Hs0[5],k=2**6-1)[0]
#print lalg.eigh(Hs0[5])[0]
# I think that this sparse code will return all of the eigenthings except for one near the centre
qubits = 8
a_n = lalg.eigh(Hs0[qubits-1])[0]
a_s =spalg.eigsh(sparse_Hs0[qubits-1],k=2**qubits-1)[0]
print len(a_n)
excluded_index = 145
difference = a_s-a_n[np.arange(len(a_n))!=excluded_index]
difference[abs(difference)<1e-13]=0
print difference.size
print difference

## From playing about with the above cell I have empirically discovered that this method will miss out an index from somewhere close to and slightly above the centre index

This should mean that it is safe for me to use.

# I have just realised that this will not work for me since I need to have all of the eigenvalues and eigenvectors in order to build the matrix unitary time evolution matrix. Shite! This could be an end of this thought chain

The only other option I can think of is the possibility of using normal matrix multiplication with sparse methods. I mean by this that we will work in the straight basis, not the diagonalised one

In [ ]:
# lets see how it would look to work in the non-diagonalised basis instead of the diagonalised one.
# remembering that we will need to diagonalise as well as do the regular caluculation
qubits = 4
dt = 0.001
#make a vector
v = np.matrix(np.random.random(2**qubits)).getH()
v=v/(v.getH()*v).item(0)**0.5

Hs = Hs0[qubits-1]
sparse_Hs =sparse_Hs0[qubits-1]

def sparse_function():
    d,P = sparse_diagal()
    new_v = spalg.expm_multiply(sparse_Hs*(-1.j*dt), v)
    return new_v,d

def normal_function():
    d,P = diagal()
    P = np.matrix(P)
    P_in = P.getH()
    new_v = P*np.diag(np.exp(-1.j*d*dt))*P_in*v
    return new_v,d

In [ ]:
nv,nd = normal_function()
sv,sd = sparse_function()
print nv-sv

In [ ]:
# lets see how the number of qubits affects the ratio in time taken

Qmax = 12
Tn = np.zeros(Qmax)
Ts = np.zeros(Qmax)
for qubits in range(3,Qmax+1):
    Hs = Hs0[qubits-1]
    sparse_Hs = sparse_Hs0[qubits-1]
    v = v0[qubits-1]
    
    Tn[qubits-1] = timeit.timeit(normal_function,number = 10)
    Ts[qubits-1] = timeit.timeit(sparse_function,number = 10)
    
plt.figure()
plt.title('Time taken for evolution step')
plt.plot(range(1,Qmax+1), Tn, label = 'normal')
plt.plot(range(1,Qmax+1), Ts, label = 'sparse')
plt.xlabel('Qubits')
plt.ylabel('Time Taken')
plt.legend(framealpha= 0, loc = 'best')
plt.show()